In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
import sys
import tensorflow as tf
import time
from tensorflow import keras
import os

print(tf.__version__)

In [ ]:
train_dir = "10-monkey-species/training/training"
valid_dir = "10-monkey-species/validation/validation"
label_file = "10-monkey-species/monkey_labels.txt"
print(os.path.exists(train_dir))
print(os.path.exists(valid_dir))
print(os.path.exists(label_file))

print(os.listdir(train_dir))
print(os.listdir(valid_dir))

## 数据处理

### 处理数据path和class

In [ ]:
#  用于将classname转成id, id转成classname
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
train_labels_file = './cifar-10/trainLabels.csv'
test_csv_file = './cifar-10/sampleSubmission.csv'
train_folder = './cifar-10/train/'
test_folder = './cifar-10/test/'

# 解析csv文件
def parse_csv_file(filepath, folder):
    results = []
    with open(filepath, 'r') as f:
        lines = f.readlines()[1:] # 读取数据并去掉header
    for line in lines:
        image_id, label_str = line.strip('\n').split(',')
        image_full_path = os.path.join(folder, image_id+'.png')
        results.append((image_full_path, label_str))
    return results
train_labels_info = parse_csv_file(train_labels_file, train_folder)
test_csv_info = parse_csv_file(test_csv_file, test_folder)

import pprint
pprint.pprint(train_labels_info[0: 5])
pprint.pprint(test_csv_info[0: 5])
print(len(train_labels_info), len(test_csv_info))

In [ ]:
# train_df = pd.DataFrame(train_labels_info)
train_df = pd.DataFrame(train_labels_info[0: 45000])
valid_df = pd.DataFrame(train_labels_info[45000:])
test_df = pd.DataFrame(test_csv_info)

train_df.columns = ['filepath', 'class']
valid_df.columns = ['filepath', 'class']
test_df.columns = ['filepath', 'class']

print(train_df.head())
print(valid_df.head())
print(test_df.head())

### image generator

In [ ]:
# 定义图片大小
height = 32
width = 32
channels = 3
batch_size = 32
num_classes = 10

# 创建数据generator（针对图片的api）
# 里面的参数都是做数据增强的
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,    # 缩放数据，将每个值除以255
                                                             rotation_range = 40, # 旋转角度，随机旋转（-40， 40）度
                                                             width_shift_range = 0.2,   # 水平位移 
                                                             height_shift_range = 0.2,  # 垂直位移
                                                             shear_range = 0.2, # 剪切强度
                                                             zoom_range = 0.2, # 缩放强度
                                                             horizontal_flip = True, # 是否随机做水平翻转
                                                             fill_mode = 'nearest'  # 填充像素策略
                                                              )
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory = './',
                                                    x_col = 'filepath', # x列列名
                                                    y_col = 'class',
                                                    classes = class_names,
                                                    target_size = (height, width),
                                                    batch_size = batch_size,
                                                    seed = 7,
                                                    shuffle = True,
                                                    class_mode = 'sparse'   # 1D整数标签
                                                    )
valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
valid_generator = valid_datagen.flow_from_dataframe(valid_df,
                                                    directory = './',
                                                    x_col = 'filepath', # x列列名
                                                    y_col = 'class',
                                                    classes = class_names,
                                                    target_size = (height, width),
                                                    batch_size = batch_size,
                                                    seed = 7,
                                                    shuffle = False,
                                                    class_mode = 'sparse' 
                                                    )

train_num = train_generator.samples  # 训练集数据数量
valid_num = train_generator.samples
print(train_num, valid_num)

In [ ]:
# 从generator中取数据
for i in range(2):
    x, y = train_generator.next()
    print(x.shape, y.shape)
    print(y)

In [ ]:
labels = pd.read_csv(label_file, header=0)
print(labels)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', input_shape=[width, height, channels]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='selu'),
    keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
epochs = 10
history = model.fit_generator(train_generator, 
                              steps_per_epoch = train_num // batch_size, 
                              epochs=epochs, 
                              validation_data=valid_generator,
                              validation_steps=valid_num // batch_size
                             )

In [ ]:
print(history.history.keys())

In [ ]:
def plot_learning_curves(history, label, epochs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()

plot_learning_curves(history, 'accuracy', epochs, 0, 1)
plot_learning_curves(history, 'loss', epochs, 0, 2.5)

In [ ]:
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,    # 缩放数据，将每个值除以255
                                                              )
test_generator = train_datagen.flow_from_dataframe(test_df,
                                                    directory = './',
                                                    x_col = 'filepath', # x列列名
                                                    y_col = 'class',
                                                    classes = class_names,
                                                    target_size = (height, width),
                                                    batch_size = batch_size,
                                                    seed = 7,
                                                    shuffle = True,
                                                    class_mode = 'sparse'   # 1D整数标签
                                                    )
test_num = test_generator.samples
print(test_num)

In [ ]:
test_predict = model.predict_generator(test_generator, 
                                       workers=10, 
                                       use_multiprocessing=True   # True: 使用workers个进程并行操作，False: 使用workers个线程
                                      )

In [ ]:
# 获取预测值的index
test_predict_class_indices = np.argmax(test_predict, axis = 1)

In [ ]:
# 获取预测值的classname
test_predict_class = [class_names[index] for index in test_predict_class_indices]

In [ ]:
def generate_submissions(filenmae, predict_class):
    with open(filename, 'w') as f:
        f.write('id, label\n')
        for i in range(len(predict_class)):
            f.write('%d, %s\n' % (i+1, predict_class[i]))
output_file = './cifar-10/submission.csv'
generate_submissions(output_file, test_predict_class)